# X-ray emission measurements using the tender X-ray emission spectrometer (TEXS)

In this exercise, you will use data measured using the TEXS from ID26. A particularity regarding data analyisys, is that the signal of each scan must be reconstructed from the individual signals of the 11 crystal analyzers. Also, the individual signals can present spikes due to cosmic rays, which must be removed. The measurments are for an IrRu(Ti) alloy.

<figure>
  <img src="assets/texs.png" alt="TEXS" style="width:70%">
  <img src="assets/texs_crystal_analyzers.png" alt="TEXS" style="width:37%">
  <figcaption style="text-align: center; font-style: italic">M. Rovezzi et al., J Synchrotron Rad 27, 813 (2020).</figcaption>
</figure>

## Import the required packages

In [ ]:
%matplotlib ipympl

import sys
import logging

import numpy as np
import matplotlib.pyplot as plt
import silx.io.h5py_utils

from daxs.filters import hampel

plt.rcParams["figure.figsize"] = (6, 3.4)

## Read the data

Start by reading the data from the HDF5 file. Note that there are multiple counters for the signal, each corresponding to the individual crystal analyzers.

In [ ]:
filename = "experimental_data/ihma176/id26/Ir47Ru46Ti7_AP/Ir47Ru46Ti7_AP_La_VtC/Ir47Ru46Ti7_AP_La_VtC.h5"

counters = {
    "x": "xes_en_texs",
    "signal": ["g02", "g03", "g04", "g05", "g06", "g07", "g11", "g12", "g13", "g14"],
    "monitor": "I0t" 
}

scan_name = "2.1"

signals = []
with silx.io.h5py_utils.File(filename) as sf:
    x = sf[f"{scan_name}/measurement/{counters['x']}"][()]
    for counter in counters["signal"]:
        signals.append(sf[f"{scan_name}/measurement/{counter}"][()])
    monitor = sf[f"{scan_name}/measurement/{counters['monitor']}"][()]

## Plot the data from the crystal analyzers

Iterate over the signals and plot them. Add the counter name of the signals to the legend.

In [ ]:
fig, ax = plt.subplots()

for i, signal in enumerate(signals):
    ##################
    # YOUR CODE HERE # 
    counter = counters["signal"][i]
    ax.plot(x, signal, label=f"{counter}")
    ##################
ax.legend()

## Detect and plot the outliers 

Use the Hampel filter implemented in the `daxs` library to detect the outliers in the signal. You can find more information about it using `hampel?`.

In [ ]:
fig, ax = plt.subplots()

for i, signal in enumerate(signals):
    counter = counters["signal"][i]
    ids, medians = hampel(signal, window_size=5, threshold=3)
    ax.plot(x, signal, label=f"{counter}")
    ax.plot(x[ids], signal[ids], 'k.')
    
ax.legend()

Find the optimal parameters for the threshold and size of the sliding window to select only the "real" outliers. Once satisfied with the selection, substitute each outlier with the median value calculated at that point. Use the `np.where` function for this.

In [ ]:
fig, ax = plt.subplots()

filtered_signals = []

for i, signal in enumerate(signals):
    counter = counters["signal"][i]
    ##################
    # YOUR CODE HERE #     
    ids, medians = hampel(signal, window_size=11, threshold=7)
    signal = np.where(ids, medians, signal)
    ##################
    filtered_signals.append(signal)    
    ax.plot(x, signal, label=f"{counter}")
    
ax.legend()

## Plot the mean filtered signal

Try to achieve this without looping over the `filtered_signals` list. Instead convert it to a Numpy array and use the `np.mean` function to sum along the rows.

In [ ]:
filtered_signals = np.array(filtered_signals)
print(filtered_signals.shape)

# Calculate the mean of the filtered signal.
##################
# YOUR CODE HERE # 
mean_filtered_signal = np.mean(filtered_signals, axis=0)
##################

fig, ax = plt.subplots()
ax.plot(x, mean_filtered_signal / monitor)